In [1]:
import numpy as np
import pandas as pd
import math
import statsmodels.tsa.stattools as statools

In [2]:
ngonye_data='./output_data/2020/'
output_data='./output_data/forecast/'

In [3]:
ngo = pd.read_csv(ngonye_data + 'ngonye_daily.csv')
ngo['Date']=pd.to_datetime(ngo['Date'],format="%Y-%m-%d")#"%d/%m/%Y")
ngo=ngo.set_index(pd.DatetimeIndex(ngo['Date']))
ngo['DateVal']=ngo.index

In [4]:
ngo = ngo[['Flow','Exceedance','DateVal','Year','Month','Day']]
ngo

,Flow,Exceedance,DateVal,Year,Month,Day
Date,,,,,,
1924-10-01,140.730461,0.999,1924-10-01,1924,10,1
1924-10-02,140.730461,0.999,1924-10-02,1924,10,2
1924-10-03,140.730461,0.999,1924-10-03,1924,10,3
1924-10-04,140.730461,0.999,1924-10-04,1924,10,4
1924-10-05,140.730461,0.999,1924-10-05,1924,10,5
...,...,...,...,...,...,...
2020-05-16,3580.983662,0.056,2020-05-16,2020,5,16
2020-05-17,3559.103900,0.057,2020-05-17,2020,5,17
2020-05-18,3490.510716,0.059,2020-05-18,2020,5,18


In [7]:
actual = pd.read_csv(output_data + 'ngonye_forecast_actual.csv')
actual['DateVal']=pd.to_datetime(actual['DateVal'],format="%Y-%m-%d")
actual=actual.set_index('DateVal')
actual = actual.rename(columns={c: int(c) for c in actual.columns})
actual

,1,2,3,4,5,6,7,8,9,10,...,21,22,23,24,25,26,27,28,29,30
DateVal,,,,,,,,,,,,,,,,,,,,,
1924-10-01,140.730461,140.730461,140.730461,140.730461,140.730461,140.730461,140.730461,140.730461,140.730461,140.730461,...,144.610705,144.610705,147.267478,136.426365,136.426365,136.426365,136.426365,136.426365,136.426365,136.426365
1924-10-02,140.730461,140.730461,140.730461,140.730461,140.730461,140.730461,140.730461,140.730461,140.730461,140.730461,...,144.610705,147.267478,136.426365,136.426365,136.426365,136.426365,136.426365,136.426365,136.426365,138.899082
1924-10-03,140.730461,140.730461,140.730461,140.730461,140.730461,140.730461,140.730461,140.730461,140.730461,140.730461,...,147.267478,136.426365,136.426365,136.426365,136.426365,136.426365,136.426365,136.426365,138.899082,141.399969
1924-10-04,140.730461,140.730461,140.730461,140.730461,140.730461,140.730461,140.730461,140.730461,140.730461,140.730461,...,136.426365,136.426365,136.426365,136.426365,136.426365,136.426365,136.426365,138.899082,141.399969,143.928906
1924-10-05,140.730461,140.730461,140.730461,140.730461,140.730461,140.730461,140.730461,140.730461,140.730461,140.730461,...,136.426365,136.426365,136.426365,136.426365,136.426365,136.426365,138.899082,141.399969,143.928906,138.842814
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2020-05-16,3559.103900,3490.510716,3390.523841,3318.867319,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-05-17,3490.510716,3390.523841,3318.867319,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-05-18,3390.523841,3318.867319,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
rmse=pd.DataFrame()

In [9]:
def calcRMSE(model,prefix):
    d1=model[np.arange(1,31,1)]-actual[np.arange(1,31,1)]
    d1['DateVal']=ngo['DateVal']
    d1['Flow']=ngo['Flow']
    e1=d1[np.arange(1,31,1)].divide(d1['Flow'],axis=0)
    rmse[prefix]=((e1[np.arange(1,31,1)]**2).mean()**0.5)
    #rmse=((e1[np.arange(1,31,1)]**2).mean()**0.5).to_frame().rename(columns={0:"linear_1p"})
    #rmse

In [10]:
m1=pd.concat([pd.DataFrame(columns=np.arange(1,31,1)),ngo])
m1.index.name='DateVal'
m1['m']=m1['Flow']-m1['Flow'].shift(1)
m1[np.arange(1,31,1)]= m1.apply(lambda x: x['Flow']+(pd.Series(np.arange(1,31,1))*x['m']),axis=1)

In [11]:
calcRMSE(m1,'linear_2p')

In [12]:
m3=pd.concat([pd.DataFrame(columns=np.arange(1,31,1)),ngo])
m3.index.name='DateVal'
m3['m']=(m3['Flow']-m3['Flow'].shift(4))/4
m3[np.arange(1,31,1)]= m3.apply(lambda x: x['Flow']+(pd.Series(np.arange(1,31,1))*x['m']),axis=1)

In [13]:
calcRMSE(m3,'linear_3p')

In [14]:
m4=pd.concat([pd.DataFrame(columns=np.arange(1,31,1)),ngo])
m4.index.name='DateVal'
m4['m']=(m4['Flow']-m4['Flow'].shift(8))/8
m4[np.arange(1,31,1)]= m4.apply(lambda x: x['Flow']+(pd.Series(np.arange(1,31,1))*x['m']),axis=1)

In [15]:
calcRMSE(m4,'linear_7p')

In [18]:
m5=pd.concat([pd.DataFrame(columns=np.arange(1,31,1)),ngo])
m5.index.name='DateVal'
m5['m']=m5['Flow'].rolling(3).apply(lambda r: pd.Series(np.polyfit([1,2,3],r,1)[0]))
m5['c']=m5['Flow'].rolling(3).apply(lambda r: pd.Series(np.polyfit([1,2,3],r,1)[1]))
m5[np.arange(1,31,1)]= m5.apply(lambda x: x['c']+(pd.Series(np.arange(4,34,1))*x['m']),axis=1)

In [19]:
calcRMSE(m5,'poly_d1_3p')

In [20]:
m6=pd.concat([pd.DataFrame(columns=np.arange(1,31,1)),ngo])
m6.index.name='DateVal'
m6['m']=m6['Flow'].rolling(7).apply(lambda r: pd.Series(np.polyfit([1,2,3,4,5,6,7],r,1)[0]))
m6['c']=m6['Flow'].rolling(7).apply(lambda r: pd.Series(np.polyfit([1,2,3,4,5,6,7],r,1)[1]))
m6[np.arange(1,31,1)]= m6.apply(lambda x: x['c']+(pd.Series(np.arange(8,38,1))*x['m']),axis=1)

In [21]:
calcRMSE(m6,'poly_d1_7p')

In [23]:
m7=pd.concat([pd.DataFrame(columns=np.arange(1,31,1)),ngo])
m7.index.name='DateVal'
m7['m']=m7['Flow'].rolling(3).apply(lambda r: pd.Series(np.polyfit([1,2,3],r,2)[0]))
m7['n']=m7['Flow'].rolling(3).apply(lambda r: pd.Series(np.polyfit([1,2,3],r,2)[1]))
m7['c']=m7['Flow'].rolling(3).apply(lambda r: pd.Series(np.polyfit([1,2,3],r,2)[2]))
m7[np.arange(1,31,1)]= m7.apply(lambda x: x['c']+(pd.Series(np.arange(4,34,1))**2*x['m'])+(pd.Series(np.arange(4,34,1))*x['n']),axis=1)

In [24]:
calcRMSE(m7,'poly_d2_3p')

In [25]:
m8=pd.concat([pd.DataFrame(columns=np.arange(1,31,1)),ngo])
m8.index.name='DateVal'
m8['m']=m8['Flow'].rolling(7).apply(lambda r: pd.Series(np.polyfit([1,2,3,4,5,6,7],r,2)[0]))
m8['n']=m8['Flow'].rolling(7).apply(lambda r: pd.Series(np.polyfit([1,2,3,4,5,6,7],r,2)[1]))
m8['c']=m8['Flow'].rolling(7).apply(lambda r: pd.Series(np.polyfit([1,2,3,4,5,6,7],r,2)[2]))
m8[np.arange(1,31,1)]= m8.apply(lambda x: x['c']+((pd.Series(np.arange(8,38,1))**2)*x['m'])+(pd.Series(np.arange(8,38,1))*x['n']),axis=1)

In [26]:
calcRMSE(m8,'poly_d2_7p')

In [29]:
m9=pd.concat([pd.DataFrame(columns=np.arange(1,31,1)),ngo])
m9.index.name='DateVal'
m9['m']=m9['Flow'].rolling(7).apply(lambda r: pd.Series(np.polyfit([1,2,3,4,5,6,7],r,3)[0]))
m9['n']=m9['Flow'].rolling(7).apply(lambda r: pd.Series(np.polyfit([1,2,3,4,5,6,7],r,3)[1]))
m9['o']=m9['Flow'].rolling(7).apply(lambda r: pd.Series(np.polyfit([1,2,3,4,5,6,7],r,3)[2]))
m9['c']=m9['Flow'].rolling(7).apply(lambda r: pd.Series(np.polyfit([1,2,3,4,5,6,7],r,3)[3]))
m9[np.arange(1,31,1)]= m9.apply(lambda x: x['c']+((pd.Series(np.arange(8,38,1))**3)*x['m'])+((pd.Series(np.arange(8,38,1))**2)*x['n'])+(pd.Series(np.arange(8,38,1))*x['o']),axis=1)

In [30]:
calcRMSE(m9,'poly_d3_7p')

In [31]:
rmse.to_csv(output_data + 'forecast_model_errors.csv')

In [32]:
def split_and_save(model,prefix):
    model['Year']=model.index.year
    model['Month']=model.index.month
    model['Day']=model.index.day
    model['WaterYear']=model.apply((lambda x: (x['Year'] if x['Month']>=10 else x['Year']-1)),axis=1)
    years=model['WaterYear'].unique().tolist()

    for year in years:
        days=model.loc[model.WaterYear==year]
        days.to_csv(output_data + prefix + '_' + str(year) + '.csv',float_format='%.0f')

In [33]:
split_and_save(m1,'linear_2p')
split_and_save(m3,'linear_3p')
split_and_save(m4,'linear_7p')
split_and_save(m5,'poly_d1_3p')
split_and_save(m6,'poly_d1_7p')
split_and_save(m7,'poly_d2_3p')
split_and_save(m8,'poly_d2_7p')
split_and_save(m9,'poly_d3_7p')

## Actual calc

In [ ]:
actual=pd.DataFrame(columns=np.arange(1,31,1))
actual['DateVal']=[]
actual

for index, row in ngo.reset_index().iterrows():

    vals=pd.Series(ngo['Flow'].iloc[index+1:index+31]).reset_index().drop('Date',axis=1)
    vals=vals.rename(columns={"Flow":row['DateVal']})
    vals.index+=1
    vals=vals.transpose().reset_index().rename(columns={"index":"DateVal"})
    if index % 1000 == 0:
        display(index)
    actual=pd.concat([actual,vals],axis=0)

actual=actual.reset_index().set_index('DateVal')
actual=actual.drop('index',axis=1) 
actual.to_csv(ngonye_data + 'ngonye_forecast_actual.csv')